In [2]:
import  random
import numpy as np
import time
import pickle
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants

In [3]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
import pandas as pd
dep_train=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XTr.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_train=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yTr.dat",sep='\s+',names=['target'])
dep_val=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XV.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_val=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yV.dat",sep='\s+',names=["target"])
dep_test=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XT.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_test=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yT.dat",sep='\s+',names=["target"])

dep_train.shape,indep_train.shape,dep_val.shape,indep_val.shape,dep_test.shape,indep_test.shape

((42039, 16), (42039, 1), (10935, 16), (10935, 1), (10934, 16), (10934, 1))

In [5]:
X_subTr = np.loadtxt('D:\\bro\\Analysis\\mSANN\\cd\\XTr.dat')# (42039, 16) 
X_subV = np.loadtxt('D:\\bro\\Analysis\\mSANN\\cd\\XV.dat') # (10935, 16) 
y_subCdTr = (np.loadtxt('D:\\bro\\Analysis\\mSANN\\cd\\yTr.dat')).reshape((-1,1)) # (42039, 1) 
y_subCdV = (np.loadtxt('D:\\bro\\Analysis\\mSANN\\cd\\yV.dat')).reshape((-1,1)) # (10935, 1) 
dy_subCdTr = np.loadtxt('D:\\bro\\Analysis\\mSANN\\cd\\dyTr.dat') # (42039, 16) 
X_transTr = np.loadtxt('D:\\bro\\Analysis\\mSANN\\cd\\X_transTr.dat') # (4120, 10) 
X_transV = np.loadtxt('D:\\bro\\Analysis\\mSANN\\cd\\X_transV.dat') # (787, 10) 
y_transCdTr = (np.loadtxt('D:\\bro\\Analysis\\mSANN\\cd\\y_transCdTr.dat')).reshape((-1,1)) # (4120, 1) 
y_transCdV = (np.loadtxt('D:\\bro\\Analysis\\mSANN\\cd\\y_transV.dat')).reshape((-1,1)) # (787, 1) 
dy_transCdTr = np.loadtxt('D:\\bro\\Analysis\\mSANN\\cd\\dy_transCdTr.dat') # (4120, 10)
print(X_subTr.shape,X_subV.shape,y_subCdTr.shape,y_subCdV.shape,dy_subCdTr.shape,X_transTr.shape,X_transV.shape,y_transCdTr.shape,y_transCdV.shape,dy_transCdTr.shape)

(42039, 16) (10935, 16) (42039, 1) (10935, 1) (42039, 16) (4120, 10) (787, 10) (4120, 1) (787, 1) (4120, 10)


In [6]:
# Set up NN parameters
INPUT_DIM = dep_train.shape[1]
OUTPUT_DIM = 1
NUM_SAMPLES =  int(dep_train.shape[0]/3)
NUM_TRAINING = dep_train.shape[0]
NUM_VALIDATING = dep_val.shape[0]
NUM_HIDDEN = 100
print(INPUT_DIM,OUTPUT_DIM,NUM_SAMPLES,NUM_TRAINING,NUM_VALIDATING,NUM_HIDDEN)

16 1 14013 42039 10935 100


In [7]:
class SobolevNetwork:
    def __init__(self, input_dim, num_hidden,init = None):
        self.input_dim = input_dim 
        self.num_hidden = num_hidden
        self.W1 = tf.Variable(tf.random_normal([self.input_dim, self.num_hidden],stddev=0.1))
        self.b1 = tf.Variable(tf.ones([self.num_hidden]))
        self.W2 = tf.Variable(tf.random_normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b2 = tf.Variable(tf.ones([self.num_hidden]))
        self.W3 = tf.Variable(tf.random_normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b3 = tf.Variable(tf.ones([self.num_hidden]))
        self.W4 = tf.Variable(tf.random_normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b4 = tf.Variable(tf.ones([self.num_hidden]))
        self.W5 = tf.Variable(tf.random_normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b5 = tf.Variable(tf.ones([self.num_hidden]))
        self.W6 = tf.Variable(tf.random_normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b6 = tf.Variable(tf.ones([self.num_hidden]))        
        self.W7 = tf.Variable(tf.random_normal([self.num_hidden, 1],stddev=0.1))
        self.b7 = tf.Variable(tf.ones([1]))
        self.weights = [(self.W1, self.b1), (self.W2, self.b2), (self.W3, self.b3),(self.W4, self.b4), (self.W5, self.b5), (self.W6, self.b6),(self.W7, self.b7)]
        

    def forward(self, X):
        #Input layer
        out = X
        #Hidden layers
        W,b = self.weights[0]
        out = tf.nn.tanh(tf.matmul(out, W) + b)
        W,b = self.weights[1]
        out = tf.nn.tanh(tf.matmul(out, W) + b)
        W,b = self.weights[2]
        out = tf.nn.sigmoid(tf.matmul(out, W) + b)
        W,b = self.weights[3]
        out = tf.nn.sigmoid(tf.matmul(out, W) + b)
        W,b = self.weights[4]
        out = tf.nn.leaky_relu(tf.matmul(out, W) + b)
        W,b = self.weights[5]
        out = tf.nn.relu(tf.matmul(out, W) + b)
        #Output layer
        W,b = self.weights[-1]
        out = tf.matmul(out, W) + b
        return out

In [8]:
NN = SobolevNetwork(INPUT_DIM, NUM_HIDDEN)

In [9]:
optimizer = tf.train.AdamOptimizer()
mae = tf.keras.losses.MeanAbsoluteError()

In [10]:
NN.fit(dep_train,indep_train)

AttributeError: 'SobolevNetwork' object has no attribute 'fit'

In [9]:
# Define tensors and operations
X = tf.placeholder(tf.float32, shape=[None, INPUT_DIM],name='X')
y = tf.placeholder(tf.float32, shape=[None,OUTPUT_DIM],name='y')
y_der = tf.placeholder(tf.float32, shape=[None, INPUT_DIM],name='dydX')
print(X.shape,y.shape,y_der.shape)

(?, 16) (?, 1) (?, 16)


In [10]:
model = SobolevNetwork(INPUT_DIM, NUM_HIDDEN) 
y_p = model.forward(X)
predict_named = tf.identity(y_p, "prediction")
dy = tf.gradients(y_p, X)
predict_named = tf.identity(dy, "gradient")
optimizer = tf.train.AdamOptimizer()
print(model,y_p)

<__main__.SobolevNetwork object at 0x000001792FE57250> Tensor("add_6:0", shape=(?, 1), dtype=float32)


In [11]:
yLossLambda = dict()
for lam in range(11):
    yLossLambda[lam] = tf.reduce_mean(tf.pow(y_p - tf.reshape(y,[NUM_SAMPLES,1]), 2) + lam/10. * tf.reshape(tf.reduce_sum(tf.pow(dy - y_der, 2),2),[NUM_SAMPLES,1]))


In [12]:
# Train storage
optim = dict()
timer = dict()
resTrain = dict()
for lam in range(11):
    optim[lam] = optimizer.minimize(yLossLambda[lam])
    timer[lam] = []
    resTrain[lam] = []

In [13]:
# Validation storage
resValid = []
resValid1 = []
resValid2 = []

# Saved model storage
saver = tf.train.Saver(max_to_keep=100)
errModel = np.zeros((1000,1000))

In [14]:
# Sobolev optimizer
tstart = time.time()
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [29]:
batch_samples = random.sample(range(NUM_TRAINING),NUM_SAMPLES)
dep_train1=dep_train.to_numpy()
indep_train1=indep_train.to_numpy()
X_train = dep_train1[batch_samples,:]
y_train = indep_train1[batch_samples,:]
train_dict = {X: X_train, y: y_train}

In [30]:
_, current_loss, ytrain_loss, ydytrain_loss = sess.run([optim[lam],yLossLambda[lam], yLossLambda[0],yLossLambda[10]], feed_dict=train_dict)

InvalidArgumentError: Graph execution error:

Detected at node 'dydX' defined at (most recent call last):
    File "c:\Users\kiran.franklin\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\kiran.franklin\Anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\kiran.franklin\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\kiran.franklin\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\kiran.franklin\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\Users\kiran.franklin\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\kiran.franklin\Anaconda3\lib\asyncio\base_events.py", line 

In [ ]:
XV = np.zeros((nTransV+nSubV,dimTrans + dimSub - 2))

In [ ]:
X_valid = XV;y_valid = yV;dy_valid = dyV
valid_dict = {X: X_valid, y: y_valid, y_der: dy_valid}    
X_valid1 = XV[:nTransV,:];y_valid1 = yV[:nTransV,:];dy_valid1 = dyV[:nTransV,:]
valid_dict1 = {X: X_valid1, y: y_valid1, y_der: dy_valid1}
X_valid2 = XV[nTransV:,:];y_valid2 = yV[nTransV:,:];dy_valid2 = dyV[nTransV:,:]
valid_dict2 = {X: X_valid2, y: y_valid2, y_der: dy_valid2}
best_sub = 1000
best_trans = 1000
iter = 0